# 市区町村のポリゴン作成

使用するデータ：[国土数値情報 \| 行政区域データ](https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-N03-v3_1.html)

上記全国版のオリジナルのファイルを用いることもできるが、以下のような問題がある。
- ファイルサイズが大きい(1GB以上)
- 一つの自治体に対して複数のフィールドを持つため、結合などの処理が面倒

上記問題を解決するため、geometryを六角形メッシュ状に簡略化し、各自治体毎に1行にまとめたGISファイルを作成した。

六角形メッシュとすることで相当程度拡大しても視認性に違和感なく、ファイルサイズを15MB程度に縮小できた。


## 行政区域データのダウンロード

全国版のデータもあるが、処理する上で読み込みに時間が掛かるため、都道府県毎にデータを取得する。

In [ ]:
RAW_PATH = 'raw/kukaku'

In [ ]:
import urllib.request as rq
import os

def download_data(n):
    name = f'{n:02}'
    
    url = f'https://nlftp.mlit.go.jp/ksj/gml/data/N03/N03-2022/N03-20220101_{name}_GML.zip'
    
    save_path = f'{RAW_PATH}/{name}.zip'
    
    if os.path.exists(save_path): return
    
    print(f'downloading {name}... ', end='')
    with open(save_path, "wb") as f:
        f.write(rq.urlopen(url).read())
        
    print("done!!")

In [ ]:
for i in range(1, 48):
    download_data(i)

## データ整形


In [ ]:
import pandas as pd
import geopandas as gpd
import h3pandas

In [ ]:
def preproc(n, resolution):
    
    df = gpd.read_file(f'{RAW_PATH}/{n:02}.zip')
    df = df[~df['N03_007'].isna()]
    
    new = []
    df["area"] = df.area
    
    for code in df["N03_007"].unique():
        new.append(
            df[df["N03_007"] == code]
            .sort_values(by="area").tail(3)
            .h3.polyfill_resample(resolution)
            .dissolve()
        )
        
    df = pd.concat(new)
    
    # 政令指定都市の場合、行政区域のデータはあるが
    # "市"としてのデータがないので作る
    new = []
    for name in df.N03_003.unique():
        if not str(name).endswith("市"): continue
        
        cdf = df[df.N03_003 == name]
        cdf.N03_004 = name
        cdf.N03_007 = f'{int(cdf.N03_007.min()) // 10 * 10:05}'
        new.append(cdf.h3.polyfill_resample(resolution).dissolve())
        
    return pd.concat([df, *new])

処理の実施。5分ほど掛かる。

ポリゴンの粒度を細かくしたい場合は`preproc`関数に渡す第2引数を`9`や`10`などにするとよい。(その分処理時間は倍増する)

In [ ]:
jp = []

for i in range(1, 48):
    jp.append(preproc(i, 8))

In [ ]:
jpdf = pd.concat(jp)

# 政令指定都の区の名前を「xx市oo区」とする
idxer = jpdf.N03_003.fillna("").str.endswith("市") & jpdf.N03_004.str.endswith("区")
jpdf.loc[idxer, "N03_004"] = jpdf[idxer].N03_003 + jpdf[idxer].N03_004

jpdf = (
    jpdf.rename(columns={
        "N03_001": "都道府県名", 
        "N03_004": "市区町村名", 
        "N03_007": "code"})
    .set_index("code")
)

## お試し描画

首都圏を描画してみる

In [ ]:
import plotly.express as px

In [ ]:
def plot(gdf):
    fig = px.choropleth_mapbox(gdf,
                               geojson=gdf.geometry,
                               locations=gdf.index,
                               hover_name="市区町村名",
                               center={"lat": 35.6, "lon": 139.75},
                               mapbox_style="open-street-map",
                               zoom=7.5)
    fig.update_layout(height=600)
    fig.show(width=100)

In [ ]:
plot(jpdf[jpdf["都道府県名"].str.contains("埼玉県|東京都|神奈川県|千葉県")])

## ファイル出力

GISファイルとして出力する。  
ここでは`geojson`ファイルにしているが、シェープファイルにすることもできる。

### 団体コードとポリゴンのみ

In [ ]:
jpdf.loc[:, ["geometry"]].to_file("sikuchoson.json", driver='GeoJSON')

### 団体コードとポリゴンと市区町村名

In [ ]:
jpdf.loc[:, ["市区町村名", "geometry"]].to_file("sikuchoson_name.json", driver='GeoJSON')

### 団体コードとポリゴンと市区町村名と都道府県名

In [ ]:
jpdf.loc[:, ["都道府県名", "市区町村名", "geometry"]].to_file("sikuchoson_name_pref.json", driver='GeoJSON')

## おまけ

市区町村名として郡が含まれているデータ(例：xx郡oo町)で団体コードを参照するためのテーブルを作成する。

In [ ]:
idxer = jpdf.N03_003.fillna("").str.endswith("郡")

In [ ]:
jpdf.loc[idxer, "市区町村名"] = jpdf[idxer].N03_003 + jpdf[idxer].市区町村名

In [ ]:
jpdf[idxer]

In [ ]:
jpdf[["都道府県名", "市区町村名"]].to_csv("地方公共団体コード.csv")